In [ ]:
# crontab 0 5 * * *

In [ ]:
import json

In [ ]:
import pandas as pd

In [ ]:
from urllib.parse import urljoin, urlparse, urlunparse

In [ ]:
import scrapy
from scrapy.loader import ItemLoader

In [ ]:
from itemloaders.processors import MapCompose, TakeFirst, Join

In [ ]:
import re

In [ ]:
import math

In [ ]:
from w3lib.html import remove_tags

In [ ]:
p_fax = re.compile('Fax:\s(\d{3,3}-\d{3,3}-\d{4,4})')
def get_fax(value):
    m = p_fax.findall(value)
    return m[0]

In [ ]:
p_name = re.compile('<h1>(The Honourable )?(The Right Honourable )?(.*)</h1>')
def get_name(value):
    m = p_name.findall(value)
    return m[0][2]

In [ ]:
class Fax(scrapy.Item):
    fax = scrapy.Field(
        input_processor=MapCompose(get_fax),
        output_processor=TakeFirst(),
    )
    name = scrapy.Field(
        input_processor=MapCompose(get_name),
        output_processor=TakeFirst(),
    )

In [ ]:
class CanadaFederalHouseFaxSpider(scrapy.Spider):
    name = 'canada_federal_house_faxes'
    domain = 'https://www.ourcommons.ca'
    start_urls = [
        ''.join([domain, '/Members/en/search'])
    ]

    def parse(self, response):
        members = set()
        for member in response.css('.ce-mip-mp-tile-container'):

            # Get the URL for the contact page.
            url = ''.join([self.domain, member.css('a::attr("href")').get().strip(), '#contact'])
            
            # Add to set to dedupe.
            members.add(url)
            
        yield from response.follow_all(members, callback=self.parse_member)

    def parse_member(self, response):
        loader = ItemLoader(item=Fax(), response=response)
        loader.add_css('fax', '.col-md-3')
        loader.add_css('name', 'body > main:nth-child(4) > div:nth-child(3) > h1:nth-child(2)')
        return loader.load_item()

In [ ]:
from scrapy.crawler import CrawlerProcess
# from scrapy.utils.project import get_project_settings
# print(repr(get_project_settings()))

process = CrawlerProcess({
#     'LOG_ENABLED': False,
    'FEEDS': {
        'fax.jsonl': {
            'format': 'jsonlines',
            'encoding': 'utf8',
        },
    },
})

process.crawl(CanadaFederalHouseFaxSpider)
process.start()

In [ ]:
f = open('fax.jsonl', 'r')
json_lines = [json.loads(line) for line in f]
f.close()

In [ ]:
df = pd.json_normalize(json_lines)
df